# Part 1

Your ferry made decent progress toward the island, but the storm came in faster than anyone expected. The ferry needs to take evasive actions!

Unfortunately, the ship's navigation computer seems to be malfunctioning; rather than giving a route directly to safety, it produced extremely circuitous instructions. When the captain uses the PA system to ask if anyone can help, you quickly volunteer.

The navigation instructions (your puzzle input) consists of a sequence of single-character actions paired with integer input values. After staring at them for a few minutes, you work out what they probably mean:
```
Action N means to move north by the given value.
Action S means to move south by the given value.
Action E means to move east by the given value.
Action W means to move west by the given value.
Action L means to turn left the given number of degrees.
Action R means to turn right the given number of degrees.
Action F means to move forward by the given value in the direction the ship is currently facing.
```
The ship starts by facing east. Only the L and R actions change the direction the ship is facing. (That is, if the ship is facing east and the next instruction is N10, the ship would move north 10 units, but would still move east if the following action were F.)

For example:
```
F10
N3
F7
R90
F11
```
These instructions would be handled as follows:

- `F10` would move the ship 10 units east (because the ship starts by facing east) to east 10, north 0.
- `N3` would move the ship 3 units north to east 10, north 3.
- `F7` would move the ship another 7 units east (because the ship is still facing east) to east 17, north 3.
- `R90` would cause the ship to turn right by 90 degrees and face south; it remains at east 17, north 3.
- `F11` would move the ship 11 units south to east 17, south 8.

At the end of these instructions, the ship's Manhattan distance (sum of the absolute values of its east/west position and its north/south position) from its starting position is 17 + 8 = 25.

Figure out where the navigation instructions lead. What is the Manhattan distance between that location and the ship's starting position?

In [1]:
import numpy as np


def get_input():
    with open('inputs/12') as f:
        return f.read().splitlines()

    
DIRECTIONS = {
    'N': np.array([1, 0]),
    'E': np.array([0, 1]),
    'S': np.array([-1, 0]),
    'W': np.array([0, -1])
}


class Ship():
    def __init__(self, facing_dir):
        self.coords = np.array([0, 0])
        self.facing = facing_dir
        
    def do(self, instruction):
        action = instruction[0]
        value = int(instruction[1:])
        if action in DIRECTIONS:
            self.move_direction(action, value)
        elif action == 'F':
            self.move_forward(value)
        elif action == 'L':
            self.turn_left(value)
        elif action == 'R':
            self.turn_right(value)
        else:
            raise Exception('Unsupported instruction: %s' % instruction)
        
    def move_direction(self, direction, distance):
        self.coords += DIRECTIONS[direction] * distance
        
    def move_forward(self, distance: int):
        self.coords += (self.facing * distance)
    
    def turn_left(self, degrees=90):
        num_turns = (degrees / 90) % 4
        if num_turns == 1:
            self._turn_left()
        if num_turns == 2:
            self.turn_back()
        if num_turns == 3:
            self._turn_right()
    
    def turn_right(self, degrees=90):
        num_turns = (degrees / 90) % 4
        if num_turns == 1:
            self._turn_right()
        if num_turns == 2:
            self.turn_back()
        if num_turns == 3:
            self._turn_left()
    
    def _turn_left(self):
        facing_x, facing_y = self.facing
        self.facing = np.array([facing_y, -facing_x])
    
    def _turn_right(self):
        facing_x, facing_y = self.facing
        self.facing = np.array([-facing_y, facing_x])
        
    def turn_back(self):
        self.facing *= -1
    
    def distance(self):
        return sum(np.absolute(self.coords))


def result1():
    instructions = get_input()
    ship = Ship(facing_dir=DIRECTIONS['E'])
    for i in instructions:
        ship.do(i)
    return ship.distance()
    
    
result1()

1631

# Part 2

Before you can give the destination to the captain, you realize that the actual action meanings were printed on the back of the instructions the whole time.

Almost all of the actions indicate how to move a waypoint which is relative to the ship's position:
```
Action N means to move the waypoint north by the given value.
Action S means to move the waypoint south by the given value.
Action E means to move the waypoint east by the given value.
Action W means to move the waypoint west by the given value.
Action L means to rotate the waypoint around the ship left (counter-clockwise) the given number of degrees.
Action R means to rotate the waypoint around the ship right (clockwise) the given number of degrees.
Action F means to move forward to the waypoint a number of times equal to the given value.
```
The waypoint starts 10 units east and 1 unit north relative to the ship. The waypoint is relative to the ship; that is, if the ship moves, the waypoint moves with it.

For example, using the same instructions as above:

- `F10` moves the ship to the waypoint 10 times (a total of 100 units east and 10 units north), leaving the ship at east 100, north 10. The waypoint stays 10 units east and 1 unit north of the ship.
- `N3` moves the waypoint 3 units north to 10 units east and 4 units north of the ship. The ship remains at east 100, north 10.
- `F7` moves the ship to the waypoint 7 times (a total of 70 units east and 28 units north), leaving the ship at east 170, north 38. The waypoint stays 10 units east and 4 units north of the ship.
- `R90` rotates the waypoint around the ship clockwise 90 degrees, moving it to 4 units east and 10 units south of the ship. The ship remains at east 170, north 38.
- `F11` moves the ship to the waypoint 11 times (a total of 44 units east and 110 units south), leaving the ship at east 214, south 72. The waypoint stays 4 units east and 10 units south of the ship.

After these operations, the ship's Manhattan distance from its starting position is 214 + 72 = 286.

Figure out where the navigation instructions actually lead. What is the Manhattan distance between that location and the ship's starting position?

In [2]:
class Ship2():
    def __init__(self, waypoint):
        self.coords = np.array([0, 0])
        self.waypoint = waypoint
        
    def do(self, instruction):
        action = instruction[0]
        value = int(instruction[1:])
        if action in DIRECTIONS:
            self.move_waypoint(action, value)
        elif action == 'F':
            self.move_forward(value)
        elif action == 'L':
            self.rotate_waypoint_left(value)
        elif action == 'R':
            self.rotate_waypoint_right(value)
        else:
            raise Exception('Unsupported instruction: %s' % instruction)
        
    def move_waypoint(self, direction, distance):
        self.waypoint += DIRECTIONS[direction] * distance
        
    def move_forward(self, distance: int):
        self.coords += (self.waypoint * distance)
    
    def rotate_waypoint_left(self, degrees=90):
        num_turns = (degrees / 90) % 4
        if num_turns == 1:
            self._rotate_waypoint_left()
        if num_turns == 2:
            self._rotate_waypoint_back()
        if num_turns == 3:
            self._rotate_waypoint_right()
    
    def rotate_waypoint_right(self, degrees=90):
        num_turns = (degrees / 90) % 4
        if num_turns == 1:
            self._rotate_waypoint_right()
        if num_turns == 2:
            self._rotate_waypoint_back()
        if num_turns == 3:
            self._rotate_waypoint_left()
    
    def _rotate_waypoint_left(self):
        wayp_x, wayp_y = self.waypoint
        self.waypoint = np.array([wayp_y, -wayp_x])
    
    def _rotate_waypoint_right(self):
        wayp_x, wayp_y = self.waypoint
        self.waypoint = np.array([-wayp_y, wayp_x])
        
    def _rotate_waypoint_back(self):
        self.waypoint *= -1
    
    def distance(self):
        return sum(np.absolute(self.coords))


def result2():
    instructions = get_input()
    ship = Ship2(waypoint=np.array([1, 10]))
    for i in instructions:
        ship.do(i)
    return ship.distance()
    
    
result2()

58606